# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
#load csv into dataframe and drop and rows with null value
city_weatherdata = "output_data/wetherpy.csv"
city_weatherdata_df= pd.read_csv(city_weatherdata)
city_weatherdata_df = city_weatherdata_df.dropna()
city_weatherdata_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Airai,-8.93,125.41,57.36,100,98,4.43,TL,1592332284
1,Tasiilaq,65.61,-37.64,35.60,92,75,18.34,GL,1592332284
2,Bredasdorp,-34.53,20.04,53.60,93,0,3.51,ZA,1592332284
3,Westport,41.14,-73.36,73.00,49,1,12.75,US,1592332284
4,Mossamedes,-15.20,12.15,72.48,62,0,3.56,AO,1592332285


In [20]:
city_weatherdata_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544 entries, 0 to 545
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        544 non-null    object 
 1   Lat         544 non-null    float64
 2   Lng         544 non-null    float64
 3   Max Temp    544 non-null    float64
 4   Humidity    544 non-null    int64  
 5   Cloudiness  544 non-null    int64  
 6   Wind Speed  544 non-null    float64
 7   Country     544 non-null    object 
 8   Date        544 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 42.5+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
#Configure gmaps with api_key
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 
locations = city_weatherdata_df[["Lat", "Lng"]].astype(float)
#Store humidity into variable humidity for gmaps weights
humidity = city_weatherdata_df["Humidity"].astype(float)

In [22]:
#Find the highest humidity in the dataset to set as maximum intensity for the heatmap
MaxHumidty = (city_weatherdata_df["Humidity"].max()).astype(float)
MaxHumidty

100.0

In [23]:
# Create a Humdiity Heatmap layer
fig = gmaps.figure()


heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=MaxHumidty,
                                 point_radius = 1)
fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
#Define a filter for the ideal weather condition cities ..note that I changed the temp specification from >70 and <80
# to > 60 and < 70 because I did not get enough data with the initial specs due to the time of the day that I generated
#the weather data in WeatherPy part 1. 
Criteria_filter = (city_weatherdata_df["Max Temp"] > 60) & (city_weatherdata_df["Max Temp"] < 70) & \
                  (city_weatherdata_df["Wind Speed"] < 10) & (city_weatherdata_df["Cloudiness"] == 0) 
#Passed loc on the df with the Criteria filter amd create a new df for ideal weather condition
idealweather_df = city_weatherdata_df.loc[Criteria_filter]
#check ideal weather df for rows with null values 
idealweather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 85 to 464
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        13 non-null     object 
 1   Lat         13 non-null     float64
 2   Lng         13 non-null     float64
 3   Max Temp    13 non-null     float64
 4   Humidity    13 non-null     int64  
 5   Cloudiness  13 non-null     int64  
 6   Wind Speed  13 non-null     float64
 7   Country     13 non-null     object 
 8   Date        13 non-null     int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 1.0+ KB


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
#Add a "Hotel Name" column to the DataFrame
hotel_df = idealweather_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
85,Karpinsk,59.77,60.01,61.41,71,0,9.04,RU,1592332361,
218,Glushkovo,51.34,34.63,65.10,86,0,9.82,RU,1592332568,
271,Lata,40.16,-8.33,66.99,71,0,8.14,PT,1592332638,
306,Shubarshi,48.59,57.19,66.74,47,0,6.93,KZ,1592332707,
319,Malanje,-9.54,16.34,64.26,35,0,2.80,AO,1592332710,


In [43]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 85 to 464
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        13 non-null     object 
 1   Lat         13 non-null     float64
 2   Lng         13 non-null     float64
 3   Max Temp    13 non-null     float64
 4   Humidity    13 non-null     int64  
 5   Cloudiness  13 non-null     int64  
 6   Wind Speed  13 non-null     float64
 7   Country     13 non-null     object 
 8   Date        13 non-null     int64  
 9   Hotel Name  13 non-null     object 
dtypes: float64(4), int64(3), object(3)
memory usage: 1.1+ KB


In [44]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
85,Karpinsk,59.77,60.01,61.41,71,0,9.04,RU,1592332361,
218,Glushkovo,51.34,34.63,65.10,86,0,9.82,RU,1592332568,
271,Lata,40.16,-8.33,66.99,71,0,8.14,PT,1592332638,
306,Shubarshi,48.59,57.19,66.74,47,0,6.93,KZ,1592332707,
319,Malanje,-9.54,16.34,64.26,35,0,2.80,AO,1592332710,


In [45]:

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "motel",
    "key": g_key
}

# Use the lat/lng we recovered to identify nearby hotels
#city included the for loop only to capture names of city with no hotel for inclusion in exception message 
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the type: "logding" with keyword "motel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request a
    hotel_address = requests.get(base_url, params=params)

    # convert to json
    hotel_address = hotel_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
    except (KeyError, IndexError):
        print(f"Missing field/result..skipping. No hotel found for |{city}")

Missing field/result..skipping. No hotel found for |Shubarshi
Missing field/result..skipping. No hotel found for |Mari-Turek


In [46]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
85,Karpinsk,59.77,60.01,61.41,71,0,9.04,RU,1592332361,Mini-Hotel Kolizei
218,Glushkovo,51.34,34.63,65.10,86,0,9.82,RU,1592332568,M&M Aparthotel
271,Lata,40.16,-8.33,66.99,71,0,8.14,PT,1592332638,Motel Eleven
306,Shubarshi,48.59,57.19,66.74,47,0,6.93,KZ,1592332707,
319,Malanje,-9.54,16.34,64.26,35,0,2.80,AO,1592332710,Palanca Hotel
372,Shimoda,34.67,138.95,64.99,82,0,1.01,JP,1592332781,Livemax Resort Amagi Yugashima
388,Matveyevka,52.15,56.18,64.40,52,0,8.95,RU,1592332783,"Gostinitsa ""Saraktash"""
393,Kamenka,51.32,42.77,66.00,53,0,1.01,RU,1592332784,Gostinitsa Luch
416,Mari-Turek,56.79,49.62,60.62,93,0,7.65,RU,1592332850,
433,Boyabat,41.47,34.77,67.95,67,0,0.87,TR,1592332854,Beyaz Otel Gerze


In [47]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 85 to 464
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        13 non-null     object 
 1   Lat         13 non-null     float64
 2   Lng         13 non-null     float64
 3   Max Temp    13 non-null     float64
 4   Humidity    13 non-null     int64  
 5   Cloudiness  13 non-null     int64  
 6   Wind Speed  13 non-null     float64
 7   Country     13 non-null     object 
 8   Date        13 non-null     int64  
 9   Hotel Name  13 non-null     object 
dtypes: float64(4), int64(3), object(3)
memory usage: 1.7+ KB


In [48]:
# NOTE: Do not change any of the code in this cell
   
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [49]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig =gmaps.figure()

#create maarker layer map and combine with heatmap previosuly created 
fig.add_layer(heatmap_layer)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))